In [1]:
import numpy as np
import pandas as pd
from sklearn import feature_extraction, linear_model, model_selection, preprocessing

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [2]:
#what does the data look like

train_df.iloc[:1,:]

#Flag to either produce results with cleaned or uncleaned data
clense_data=True

In [3]:
# what do disaster tweets looke like

disasters = train_df[train_df["target"]==1]
disasters.iloc[:2,:]

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1


In [4]:
#what do non disaster tweets look like
print ("non_disaster tweet")
non_disasters = train_df[train_df["target"]==0]
non_disasters.iloc[:2,:]

non_disaster tweet


,id,keyword,location,text,target
15,23,NaN,NaN,What's up man?,0
16,24,NaN,NaN,I love fruits,0


In [5]:
#clense data is so instructed, to test whether cleaning data improves performance
#lets clean the data
if (clense_data==True):
    %run DataClenser.ipynb
    train_df['text'] = train_df['text'].apply(lambda x: text_clense(x))

tää on emiserkki: https://dillindallin.com
Huba haba
What shall we do today zequestion
correct me please
haa hee ja jee  


In [6]:
count_vectorizer = feature_extraction.text.CountVectorizer()

In [7]:
train_vectors = count_vectorizer.fit_transform(train_df["text"])
test_vectors = count_vectorizer.transform(test_df["text"])

In [8]:
from sklearn.feature_extraction.text import TfidfTransformer
#tfidf = TfidfVectorizer(min_df=2, max_df=0.5, ngram_range=(1, 2))
tfidf_transformer = TfidfTransformer()
train_tfidf = tfidf_transformer.fit_transform(train_vectors)
train_tfidf.shape


(7613, 17798)

In [9]:

from sklearn.linear_model import SGDClassifier

#classifier = SGDClassifier(loss='hinge', penalty='l2',
#                         alpha=1e-3, random_state=42,
#                         max_iter=5, tol=None)
text_clf = SGDClassifier().fit(train_tfidf, train_df["target"])

train_df["target"]

0       1
1       1
2       1
3       1
4       1
       ..
7608    1
7609    1
7610    1
7611    1
7612    1
Name: target, Length: 7613, dtype: int64

In [10]:
scores = model_selection.cross_val_score(text_clf, train_vectors, train_df["target"], cv=3, scoring="f1")
scores

array([0.5979269 , 0.56410256, 0.58805513])

In [11]:
#scores = model_selection.cross_val_score(tfidf, train_vectors, train_df["target"], cv=3, scoring="f1")
#scores
predicted = text_clf.predict(test_vectors)


In [12]:
sample_submission = pd.read_csv("sample_submission.csv")
sample_submission["target"] = text_clf.predict(test_vectors)
sample_submission.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1


In [13]:
if (clense_data==True):
    sample_submission.to_csv("submission3_cleaned.csv", index=False)
else:
    sample_submission.to_csv("submission3.csv", index=False)

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...
